### **1) Installing OpenAI**

In [ ]:
#Installing OpenAI
!pip install openai
!pip install striprtf
!pip install python-docx
!install tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.5/325.5 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 4.7 MB/s eta 0:00:00
install: missing destination file operand after 'tqdm'
Try 'install --help' for more information.


### **2) Importing the necessary packages**

In [ ]:
# Import the necessary libraries
import openai
from google.colab import drive
import os
from striprtf.striprtf import rtf_to_text
from docx import Document
from tqdm import tqdm


### 3) Choosing the Model and API Key

In [ ]:
OPEN_AI_API_KEY = ""
OPEN_AI_MODEL = "gpt-4o"

### 4) Dealing with path, folder, and files

In [ ]:
# Function to convert RTF to plain text using striprtf
def convert_rtf_to_text(rtf_file_path):
    with open(rtf_file_path, 'r', encoding='utf-8') as file:
        rtf_content = file.read()
    plain_text = rtf_to_text(rtf_content)
    return plain_text

# Mount Google Drive
drive.mount('/content/drive')

# Directory containing both the RTF files and the Word file with the main prompt
directory_path = '/content/drive/My Drive/Navid_trying API/'

# Path to your Word file containing the main prompt
word_file_path = os.path.join(directory_path, 'prompt_stem.docx')  # Update this filename

# Read the main prompt directly
doc = Document(word_file_path)
main_prompt = "\n".join([para.text for para in doc.paragraphs])

# List all RTF files in the directory
rtf_files = [f for f in os.listdir(directory_path) if f.endswith('.rtf')]

# Prepare a list to collect results for the CSV
results = []

# Loop through each RTF file, convert it to plain text, and generate a completion
for rtf_file in tqdm(rtf_files, desc="Processing"):
    rtf_file_path = os.path.join(directory_path, rtf_file)
    # Extract plain text from the RTF file
    rtf_text = convert_rtf_to_text(rtf_file_path)
    # Concatenate the main prompt with the RTF text
    full_prompt = main_prompt + "\n\n" + rtf_text
    # Generate a completion using OpenAI API
    response = openai.chat.completions.create(
        model="gpt-4",
        messages=[
            {"role": "user", "content": full_prompt},
        ],
    )
    # Extract the generated text
    generated_text = response.choices[0].message.content.strip()
    # Add the results to the list
    results.append([rtf_file, full_prompt, generated_text])

# Save the results to a CSV file
df = pd.DataFrame(results, columns=['File Name', 'Full Prompt', 'Generated Response'])
df.to_csv(os.path.join(directory_path, 'results.csv'), index=False)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Processing: 100%|██████████| 15/15 [02:35<00:00, 10.35s/it]
